In [ ]:
# 필요한 라이브러리 가져오기
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
# 대상 사이트 : investing.com
url = "https://www.investing.com/news/stock-market-news"

# 프로토콜 status 확인
# 200이면 정상
resp = requests.get(url)
print(resp)

<Response [200]>


In [ ]:
import pandas as pd

# csv 만들리 위해 DataFrame 사용하기
sample = [('sampleTitle','sampleDate','samplePublisher')]
df = pd.DataFrame(sample, columns=['Title','Date','Publisher'])

In [ ]:
# string date to datetime
# stirng 날짜를 datetime으로 바꾸기 
# 현재 데이터 값 예시: Sep 15, 2022 
from datetime import datetime

In [ ]:
# 링크가 주어지면, 기사 제목과 본문을 크롤링하는 코드
def get_news_by_url(url):

    # 각자 본인의 User-Agent 정보 입력 
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.content, 'html.parser')

    news_list = []

    # 뉴스 기사 가져오기 
    # Article 태그의 js-article-item클래스만 가져오기 
    for i in bs.find('section',id="leftColumn").find_all('article', "js-article-item"):
      # i['class'] => i의 클래스 이름 가져오기
      # 스폰서 광고 기사는 없애기
      if "sponsoredArticle" in i['class']:
        continue
        
      else:
        # 댓글이 없으면 무시
        if i.find('a', 'js-scrollto-comment') == None:
          continue

        # 댓글이 있으면
        else:
          # 댓글수가 5보다 많은 기사만 가져오기
          if int(i.find('a', 'js-scrollto-comment').text) > 5:

            #출판사 구하기
            span = i.find('span', 'articleDetails').text
            x = span.split("-")
            publisher = x[0][3:]
            #쓸데 없는 빈칸 없애기
            publisher = publisher[:-1]

            #기사 제목 구하기
            title = i.find('a', 'title').text

            #기사 날짜 구하기
            #Sep 15, 2022 
            dateOutput = i.find('span', 'date').text
            date = dateOutput[3:]
            date = datetime.strptime(date, '%b %d, %Y')

            news = [title, date, publisher]

            news_list.append(news)

    return news_list

In [ ]:
for i in range(10,5000):
  url = 'https://www.investing.com/news/stock-market-news/' + str(i)

  list = get_news_by_url(url)
  # output이 존재한다면 (없을수도있으니까)
  if list:
    print("Yes, we have data in", i)
    for j in list:
      # 행 추가하기 
      df = df.append({'Title': j[0], 'Date':j[1], 'Publisher':j[2] }, ignore_index=True)


KeyboardInterrupt: ignored

In [ ]:
# csv파일로 저장하기
df.to_csv('investingCom.csv', sep=',')

In [ ]:
df

,Title,Date,Publisher
0,sampleTitle,sampleDate,samplePublisher
1,McDonald's ordered to face Byron Allen's $10 b...,2022-09-20 00:00:00,Reuters
2,Stock Market Today: Dow Slides as Focus Shifts...,2022-09-20 00:00:00,Investing.com
3,"Big bank CEOs to sell lawmakers on relief, div...",2022-09-20 00:00:00,Reuters
4,Elon Musk faces skeptics as Tesla gets ready t...,2022-09-20 00:00:00,Reuters
...,...,...,...
2681,U.S. trade chief Tai says getting 'traction' w...,2021-11-10 00:00:00,Reuters
2682,Wall Street ends lower as economic data raises...,2021-11-10 00:00:00,Reuters
2683,"Stocks slip, dollar gains as US CPI sparks tig...",2021-11-10 00:00:00,Reuters
2684,S&P 500 Closes Lower as Soaring Inflation Rein...,2021-11-10 00:00:00,Investing.com
